In [1]:
from models.Detecor import Detector
import os

weights_path = 'models/weights/'

model = Detector(path_to_weights=weights_path, path_to_tmp='tmp_files', weights_name='detector_weights_v1.pt')

In [2]:
path_to_images = './tmp_files'

image_paths = [os.path.join(path_to_images, filename) for filename in os.listdir(path_to_images)]
predicts = model.predict(path_to_images)

Detection...

image 1/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/1 (11).jpg: 384x640 7 adjs, 55.1ms
image 2/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/4 (111).jpg: 384x640 3 adjs, 1 geo, 45.2ms
image 3/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/5 (115).jpg: 384x640 1 adj, 4 geos, 48.9ms
image 4/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/5 (200).jpg: 384x640 6 adjs, 42.4ms
image 5/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/6 (71).jpg: 640x384 (no detections), 42.3ms
image 6/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/7 (160).jpg: 640x384 1 adj, 1 geo, 42.7ms
image 7/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/9 (20).jpg: 640x384 1 geo, 42.1ms
image 8/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/9 (30).jpg: 640x384 1 geo, 43.5ms
image 9/9 /Users/kuprik01/Projects/Hackathon-Atomic2024/backend/tmp_files/9 (60).jp

In [3]:
def format_output(predicts):
    predicts

[{'predict_img_path': 'tmp_filestmp_2024-06-15 13:11:52.036761.jpg',
  'names': ['adj', 'adj', 'adj', 'adj', 'adj', 'adj', 'adj'],
  'confs': [0.6270024180412292,
   0.6241234540939331,
   0.6122649312019348,
   0.4597775340080261,
   0.3671072721481323,
   0.28403645753860474,
   0.27125605940818787],
  'coords': array([[     708.05,      820.96,      748.98,      866.18],
         [     710.75,      365.87,      753.01,      407.47],
         [     397.04,      297.96,      457.77,      358.13],
         [     717.29,      436.97,       779.5,      521.51],
         [     781.44,      639.59,      823.39,      689.22],
         [     84.191,      585.01,      124.09,      636.08],
         [     108.25,       442.1,      190.08,       526.9]], dtype=float32),
  'xywhn': array([[    0.37944,     0.78108,    0.021321,    0.041875],
         [    0.38119,     0.35803,     0.02201,    0.038526],
         [    0.22261,     0.30375,    0.031632,    0.055717],
         [    0.38979,     0.4